# 🔧 CARRIER CONTROL — Debug & Testing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 06_AIRCRAFT_CARRIER — CONTROL PANEL                  ║
║                  Debug • Test RIFE • Inspect • Preview                       ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Objectif**: Tester individuellement chaque composant du pipeline CARRIER.

---

## 📦 Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/06_AIRCRAFT_CARRIER"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

# Add codebase to path
import sys
sys.path.insert(0, CODEBASE)

print(f"✓ Drive monté")
print(f"✓ Unit Root: {UNIT_ROOT}")

In [ ]:
# Install dependencies
!apt install -y ffmpeg
!pip install -q torch torchvision opencv-python-headless scipy Pillow tqdm

## 🔍 Test 1: Vérification des Chemins

In [ ]:
from pathlib import Path

paths_to_check = [
    ("Unit Root", UNIT_ROOT),
    ("Codebase", CODEBASE),
    ("IN_COMPONENTS", f"{UNIT_ROOT}/IN_COMPONENTS"),
    ("OUT_FINAL", f"{UNIT_ROOT}/OUT_FINAL"),
    ("AI Models", f"{DRIVE_ROOT}/EXODUS_AI_MODELS"),
    ("RIFE Model", f"{DRIVE_ROOT}/EXODUS_AI_MODELS/rife"),
]

print("=== Vérification Chemins ===")
for name, path in paths_to_check:
    exists = Path(path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")

In [ ]:
# Lister les fichiers disponibles dans IN_COMPONENTS
import os

components_dir = Path(f"{UNIT_ROOT}/IN_COMPONENTS")

print("\n=== Fichiers IN_COMPONENTS ===")
if components_dir.exists():
    exr_files = list(components_dir.glob("*.exr"))
    png_files = list(components_dir.glob("*.png"))
    wav_files = list(components_dir.glob("*.wav"))
    json_files = list(components_dir.glob("*.json") | components_dir.glob("*.JSON"))
    
    print(f"\n📁 Images EXR: {len(exr_files)}")
    for f in exr_files[:5]:
        print(f"   - {f.name}")
    if len(exr_files) > 5:
        print(f"   ... et {len(exr_files) - 5} autres")
    
    print(f"\n📁 Images PNG: {len(png_files)}")
    for f in png_files[:5]:
        print(f"   - {f.name}")
    if len(png_files) > 5:
        print(f"   ... et {len(png_files) - 5} autres")
    
    print(f"\n🎵 Audio WAV: {len(wav_files)}")
    for f in wav_files:
        print(f"   - {f.name}")
    
    print(f"\n📋 JSON: {len(json_files)}")
    for f in json_files:
        print(f"   - {f.name}")
else:
    print("Dossier non trouvé")

## 📦 Test 2: FFmpeg

In [ ]:
# Vérifier FFmpeg
import subprocess

result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
print("=== FFmpeg ===")
print(result.stdout.split('\n')[0])

# Vérifier codecs
result = subprocess.run(["ffmpeg", "-encoders"], capture_output=True, text=True)
encoders = result.stdout

print("\n=== Codecs Disponibles ===")
print(f"  libx265 (H.265): {'✓' if 'libx265' in encoders else '✗'}")
print(f"  libx264 (H.264): {'✓' if 'libx264' in encoders else '✗'}")
print(f"  prores_ks (ProRes): {'✓' if 'prores_ks' in encoders else '✗'}")
print(f"  aac (Audio): {'✓' if 'aac' in encoders else '✗'}")

## 🎬 Test 3: Sequence Assembler

In [ ]:
from sequence_assembler import SequenceAssembler

assembler = SequenceAssembler(verbose=True)

# Test détection pattern
test_files = sorted(components_dir.glob("*.png"))[:10]
if test_files:
    seq_info = assembler.detect_sequence_pattern(test_files)
    if seq_info:
        print("\n=== Sequence Pattern ===")
        print(f"  Pattern: {seq_info.pattern}")
        print(f"  Frames: {seq_info.first_frame} → {seq_info.last_frame}")
        print(f"  Count: {seq_info.frame_count}")
        print(f"  Resolution: {seq_info.width}x{seq_info.height}")
        print(f"  Format: {seq_info.format}")
else:
    print("Aucune image trouvée pour le test")

## 🎵 Test 4: Audio Sync

In [ ]:
from audio_sync import AudioSync

audio_sync = AudioSync(verbose=True)

# Lister les pistes audio
audio_files = list(components_dir.glob("*.wav"))

print("=== Pistes Audio ===")
for audio_path in audio_files:
    info = audio_sync.get_audio_info(audio_path)
    track_type = audio_sync.detect_track_type(audio_path)
    
    if info:
        print(f"\n🎵 {audio_path.name}")
        print(f"   Type: {track_type}")
        print(f"   Duration: {info['duration']:.2f}s")
        print(f"   Sample Rate: {info['sample_rate']} Hz")
        print(f"   Channels: {info['channels']}")

if not audio_files:
    print("Aucune piste audio trouvée")

## ⚡ Test 5: RIFE Interpolator

In [ ]:
from rife_interpolator import RIFEInterpolator

# Vérifier PyTorch et GPU
try:
    import torch
    print("=== PyTorch ===")
    print(f"  Version: {torch.__version__}")
    print(f"  CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
        print(f"  VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
except ImportError:
    print("PyTorch non installé")

In [ ]:
# Test RIFE model
RIFE_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/rife"

rife = RIFEInterpolator(model_path=RIFE_PATH, verbose=True)

print(f"\n=== RIFE Status ===")
print(f"  Model Path: {RIFE_PATH}")
print(f"  Available: {rife._rife_available}")

if not rife._rife_available:
    print("\n⚠ RIFE non disponible. Options:")
    print("  1. Télécharger RIFE: https://github.com/hzwer/Practical-RIFE")
    print(f"  2. Placer les modèles dans: {RIFE_PATH}")
    print("  3. FFmpeg minterpolate sera utilisé comme fallback")

## 📐 Test 6: Upscaler

In [ ]:
from upscaler import Upscaler

ESRGAN_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/realesrgan/realesr-general-x4v3.pth"

upscaler = Upscaler(model_path=ESRGAN_PATH, verbose=True)

print("=== Upscaler Status ===")
print(f"  Model Path: {ESRGAN_PATH}")
print(f"  Real-ESRGAN disponible: {upscaler._esrgan_available}")

if not upscaler._esrgan_available:
    print("\n⚠ Real-ESRGAN non disponible. Options:")
    print("  1. pip install basicsr realesrgan")
    print("  2. Télécharger modèle: https://github.com/xinntao/Real-ESRGAN")
    print("  3. FFmpeg Lanczos sera utilisé comme fallback")

## 🎞 Test 7: Final Encoder

In [ ]:
from final_encoder import FinalEncoder

encoder = FinalEncoder(verbose=True)

print("=== Codecs Supportés ===")
for codec, config in encoder.CODEC_PRESETS.items():
    print(f"  {codec}: {config['codec']}")

## 🚀 Test 8: Dry Run Pipeline Complet

In [ ]:
# Test dry-run du script principal
!python {CODEBASE}/EXO_06_CARRIER.py --help

In [ ]:
# Dry-run (validation chemins sans exécution)
!python {CODEBASE}/EXO_06_CARRIER.py \
    --drive-root {DRIVE_ROOT} \
    --project-name "TEST_PROJECT" \
    --dry-run -v

## 📋 Inspection PRODUCTION_PLAN.JSON

In [ ]:
import json

plan_path = Path(f"{UNIT_ROOT}/IN_COMPONENTS/PRODUCTION_PLAN.JSON")

if plan_path.exists():
    with open(plan_path) as f:
        plan = json.load(f)
    
    print("=== PRODUCTION_PLAN.JSON ===")
    print(f"\nOutput Config:")
    output = plan.get('output', {})
    print(f"  Resolution: {output.get('resolution', 'N/A')}")
    print(f"  Framerate: {output.get('framerate', 'N/A')}")
    print(f"  Codec: {output.get('codec', 'N/A')}")
    print(f"  Audio Tracks: {output.get('audio_tracks', [])}")
    
    print(f"\nScenes: {len(plan.get('scenes', []))}")
    for scene in plan.get('scenes', [])[:3]:
        print(f"  - {scene.get('scene_id', 'N/A')}")
else:
    print("PRODUCTION_PLAN.JSON non trouvé")
    print("\nExemple de structure:")
    example = {
        "output": {
            "resolution": "4K",
            "framerate": 120,
            "codec": "h265",
            "audio_tracks": ["music.wav", "sfx.wav"]
        },
        "scenes": []
    }
    print(json.dumps(example, indent=2))

---
## ✅ Checklist Debug

- [ ] Chemins vérifiés
- [ ] FFmpeg installé avec codecs requis
- [ ] Séquences images détectées
- [ ] Audio chargé
- [ ] RIFE disponible (ou fallback OK)
- [ ] Upscaler fonctionnel
- [ ] Dry-run passe